<PRE>
Федеральное государственное образовательное бюджетное учреждение высшего образования
           Финансовый университет при Правительстве Российской Федерации
                            (Финансовый университет)

                  Департамент анализа данных и машинного обучения
    
     Дисциплина «Инструментальная поддержка арнализа финансово-экономических данных»
</Pre>

13.02.2021. Семинар. Тема 1. Знакомство с современным стеком ИТ анализа данных

ПМ18-1, ПМ18-2, ПМ18-3, ПМ18-4

Учебный пример "USD-JPY".

Задача. Имеется файл с историей курса йены к доллару за 2020 год. Данные о дате и о цене находятся в одном столбце. Необходимо разделить данные на два столбца, отдельно для даты и для цены.

Шаг 1. Импорт csv-файла

In [1]:
import pandas as pd
df=pd.read_csv('USD-JPY.txt',header=None,skiprows=2,names=['Story'])
df

,Story
0,Дек 31
1,103.11
2,Дек 30
3,103.47
4,Дек 29
...,...
738,108.47
739,Янв 02
740,108.69
741,Янв 01


Шаг 2. Удалим все строки, не содержащие данных, с помощью фильтра

In [2]:
df=df[~df['Story'].str.match('История')]
df.head()

,Story
0,Дек 31
1,103.11
2,Дек 30
3,103.47
4,Дек 29


Шаг 3. Создание серий для даты и цены

In [3]:
Date=pd.Series(df['Story'][::2])
Price=pd.Series(df['Story'][1::2])

Шаг 4. Создание таблицы

In [4]:
currency=pd.DataFrame({'Date':Date,'Price':Price})
currency.head()

,Date,Price
0,Дек 31,NaN
1,NaN,103.11
2,Дек 30,NaN
3,NaN,103.47
4,Дек 29,NaN


Шаг 5. Сдвиг столбца цены на одно значение вверх

In [5]:
currency['Price']=currency['Price'].shift(-1)
currency.head()

,Date,Price
0,Дек 31,103.11
1,NaN,NaN
2,Дек 30,103.47
3,NaN,NaN
4,Дек 29,103.60


Шаг 6. Исключаем каждую вторую строку

In [6]:
currency=currency[::2]
currency.index=range(len(currency))
currency

,Date,Price
0,Дек 31,103.11
1,Дек 30,103.47
2,Дек 29,103.60
3,Дек 28,103.58
4,Дек 27,103.63
...,...,...
361,Янв 05,108.12
362,Янв 04,108.17
363,Янв 03,108.47
364,Янв 02,108.69


После того, как данные разделены, их надо привести к соответствующему типу: дату к типу `datetime`, цену к числовому типу `float`.

Шаг 7а. Преобразование даты к типу `datetime`

1 способ. С помощью словаря подстановок

In [7]:
mo_dict={'Янв':'01','Фев':'02','Мар':'03','Апр':'04',
         'Май':'05','Июн':'06','Июл':'07','Авг':'08',
         'Сен':'09','Окт':'10','Ноя':'11','Дек':'12'}

In [8]:
currency['Date']='2020 '+currency['Date'].apply(lambda x: mo_dict[x[:3]])+' '+currency['Date'].str[-2:]
currency.head()

,Date,Price
0,2020 12 31,103.11
1,2020 12 30,103.47
2,2020 12 29,103.60
3,2020 12 28,103.58
4,2020 12 27,103.63


In [9]:
currency['Date']=pd.to_datetime(currency['Date'])
currency.head()

,Date,Price
0,2020-12-31,103.11
1,2020-12-30,103.47
2,2020-12-29,103.60
3,2020-12-28,103.58
4,2020-12-27,103.63


Шаг 7б. Преобразование даты к типу datetime

2 способ. С помощью локали и `strptime()`. Необходимо повторить шаги 1-6, шаг 7а не выполнять.

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, '')
print(locale.getlocale())

In [ ]:
from datetime import datetime

In [ ]:
currency['Date']=currency['Date'].apply(lambda x: datetime.strptime('2020 '+x,'%Y %b %d'))
currency.head()

Шаг 8. Преобразование цены к числовому типу

In [ ]:
currency['Price']=pd.to_numeric(currency['Price'])
currency.head()

In [ ]:
currency.info()

### II вариант решения задачи

Этот вариант копирует подход, реализованный в Power Query, в котором на одном из шагов создается столбуц остатков от деления индексного стольбца на два.

In [79]:
df=pd.read_csv('USD-JPY.txt',header=None,skiprows=2,names=['Story'])
df=df[~df['Story'].str.match('История')]
df.reset_index(inplace=True)
df['index']=range(df.shape[0])
df.head()

,index,Story
0,0,Дек 31
1,1,103.11
2,2,Дек 30
3,3,103.47
4,4,Дек 29


In [80]:
df['index']=df['index']%2
df.head()

,index,Story
0,0,Дек 31
1,1,103.11
2,0,Дек 30
3,1,103.47
4,0,Дек 29


In [81]:
df['Date']=df[df['index']==0]['Story']
df['Price']=df[df['index']==1]['Story']
df

,index,Story,Date,Price
0,0,Дек 31,Дек 31,NaN
1,1,103.11,NaN,103.11
2,0,Дек 30,Дек 30,NaN
3,1,103.47,NaN,103.47
4,0,Дек 29,Дек 29,NaN
...,...,...,...,...
727,1,108.47,NaN,108.47
728,0,Янв 02,Янв 02,NaN
729,1,108.69,NaN,108.69
730,0,Янв 01,Янв 01,NaN


In [82]:
df['Date'].ffill(inplace=True)
df.head()

,index,Story,Date,Price
0,0,Дек 31,Дек 31,NaN
1,1,103.11,Дек 31,103.11
2,0,Дек 30,Дек 30,NaN
3,1,103.47,Дек 30,103.47
4,0,Дек 29,Дек 29,NaN


In [83]:
df.dropna(axis=0,subset=['Price'],inplace=True)
df.reset_index()
df.head()

,index,Story,Date,Price
1,1,103.11,Дек 31,103.11
3,1,103.47,Дек 30,103.47
5,1,103.60,Дек 29,103.60
7,1,103.58,Дек 28,103.58
9,1,103.63,Дек 27,103.63


In [72]:
df.drop(['index','Story'],axis=1,inplace=True)
df

,Date,Price
1,Дек 31,103.11
3,Дек 30,103.47
5,Дек 29,103.60
7,Дек 28,103.58
9,Дек 27,103.63
...,...,...
723,Янв 05,108.12
725,Янв 04,108.17
727,Янв 03,108.47
729,Янв 02,108.69


In [78]:
df=df[::-1]
df.reset_index(inplace=True,drop=True)
df

,index,Date,Price
0,1,Дек 31,103.11
1,3,Дек 30,103.47
2,5,Дек 29,103.60
3,7,Дек 28,103.58
4,9,Дек 27,103.63
...,...,...,...
361,723,Янв 05,108.12
362,725,Янв 04,108.17
363,727,Янв 03,108.47
364,729,Янв 02,108.69
